<div style="
  border-left: 6px solid #f4c430;
  background: #fffbea;
  padding: 14px 18px;
  border-radius: 10px;
  margin: 12px 0;
  font-family: Arial, sans-serif;
  box-shadow: 0 3px 8px rgba(0,0,0,0.05);
">

  <h3 style="margin:0; color:#b08a00;">🥇 Gold Layer</h3>

  <p style="margin:6px 0 0; color:#444;">
    Business-ready data layer. Aggregated, enriched,
    and optimized for reporting and dashboards.
  </p>

</div>


In [21]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Load in the clean silver dataframe

In [22]:
df_gold = pd.read_csv("../Data/silver.csv")
print(df_gold.head())

gold = df_gold.copy()

              date_cet  belgium_be  denmark_dk  germany_de  great_britain_gb  \
0  2026-02-10 14:00:00      103.19      118.02      116.22            108.68   
1  2026-02-10 15:00:00      112.55      130.20      128.12            111.38   
2  2026-02-10 16:00:00      110.50      131.15      127.66            113.96   
3  2026-02-10 17:00:00      119.26      138.66      135.24            120.55   
4  2026-02-10 18:00:00      121.80      136.04      131.71            123.04   

   great_britain_gb__epex  great_britain_gb__nordpool  netherlands_nl  \
0                  108.28                      108.84          109.12   
1                  111.38                      111.38          119.64   
2                  114.71                      113.67          118.42   
3                  120.56                      120.55          126.72   
4                  123.09                      123.03          126.35   

   norway_no  
0     113.17  
1     124.07  
2     124.14  
3     131.54  
4    

In [23]:
gold["date_cet"] = pd.to_datetime(gold["date_cet"], errors="coerce")
gold = gold.dropna(subset=["date_cet"]).sort_values("date_cet")

# Ensure numeric price columns
price_cols = [c for c in gold.columns if c != "date_cet"]
gold[price_cols] = gold[price_cols].apply(pd.to_numeric, errors="coerce")

# Make index for resampling/plots
gold_ts = gold.set_index("date_cet").sort_index()


In [24]:
# --- 1) Basic cleaning / typing
gold["date_cet"] = pd.to_datetime(gold["date_cet"], errors="coerce")
gold = gold.dropna(subset=["date_cet"]).sort_values("date_cet")

price_cols = [c for c in gold.columns if c != "date_cet"]
gold[price_cols] = gold[price_cols].apply(pd.to_numeric, errors="coerce")

gold_ts = gold.set_index("date_cet").sort_index()

# --- 2) Overview tables
overview = pd.DataFrame({
    "rows": [len(gold_ts)],
    "cols": [gold_ts.shape[1]],
    "start": [gold_ts.index.min()],
    "end": [gold_ts.index.max()],
    "markets": [len(price_cols)]
})
display(overview)

missing = gold_ts.isna().mean().sort_values(ascending=False)
display(pd.DataFrame({"missing_share": missing}).head(20))

display(gold_ts.head())

# --- 3) Choose a default market (NL if exists, else first)
market = "netherlands_nl" if "netherlands_nl" in gold_ts.columns else gold_ts.columns[0]

# --- 4) Time windows
end_time = gold_ts.index.max()
start_7d = end_time - pd.Timedelta(days=7)
last_7d = gold_ts.loc[gold_ts.index >= start_7d]

today_date = end_time.date()
today = gold_ts.loc[gold_ts.index.date == today_date]

# --- 5) Plots

# 5a) Hourly last 7 days (selected market)
if not last_7d.empty and market in last_7d.columns:
    plt.figure()
    plt.title(f"{market} — last 7 days (hourly)")
    plt.xlabel("datetime (CET)")
    plt.ylabel("EUR/MWh")
    plt.plot(last_7d.index, last_7d[market])
    plt.xticks(rotation=30, ha="right")
    plt.tight_layout()
    plt.show()

# 5b) Today's hourly curve (selected market)
if not today.empty and market in today.columns:
    plt.figure()
    plt.title(f"{market} — today ({today_date}) hourly")
    plt.xlabel("datetime (CET)")
    plt.ylabel("EUR/MWh")
    plt.plot(today.index, today[market])
    plt.xticks(rotation=30, ha="right")
    plt.tight_layout()
    plt.show()

# 5c) Daily averages — plot only top 6 markets by data availability (keeps chart readable)
daily_avg = gold_ts.resample("D").mean()
if not daily_avg.empty:
    # pick markets with most non-null values
    top_markets = gold_ts[price_cols].count().sort_values(ascending=False).head(6).index.tolist()

    plt.figure()
    plt.title("Daily average price — top markets (by coverage)")
    plt.xlabel("date")
    plt.ylabel("EUR/MWh")
    for col in top_markets:
        plt.plot(daily_avg.index, daily_avg[col], label=col)
    plt.xticks(rotation=30, ha="right")
    plt.legend(loc="best", fontsize=8)
    plt.tight_layout()
    plt.show()

# 5d) Distribution histogram (selected market)
series = gold_ts[market].dropna() if market in gold_ts.columns else pd.Series(dtype=float)
if not series.empty:
    plt.figure()
    plt.title(f"{market} — price distribution")
    plt.xlabel("EUR/MWh")
    plt.ylabel("count")
    plt.hist(series.values, bins=40)
    plt.tight_layout()
    plt.show()

# 5e) Boxplot across all markets (great overview)
if len(price_cols) >= 2:
    plt.figure()
    plt.title("Price distribution by market (boxplot)")
    plt.xlabel("market")
    plt.ylabel("EUR/MWh")
    gold_ts[price_cols].plot(kind="box", rot=90)  # pandas uses matplotlib under the hood
    plt.tight_layout()
    plt.show()

# 5f) Correlation heatmap (matplotlib)
if len(price_cols) >= 2:
    corr = gold_ts[price_cols].corr()
    plt.figure()
    plt.title("Market correlation (hourly prices)")
    plt.imshow(corr.values, aspect="auto")
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.index)), corr.index)
    plt.colorbar(label="corr")
    plt.tight_layout()
    plt.show()

# --- 6) Tables: cheapest / most expensive hours "today" for selected market
if not today.empty and market in today.columns:
    n = 5
    cheapest = today[[market]].nsmallest(n, market).copy()
    priciest = today[[market]].nlargest(n, market).copy()

    display(pd.DataFrame({
        "today_date": [str(today_date)],
        "selected_market": [market],
        "today_avg_eur_per_mwh": [float(today[market].mean())],
        "today_min_eur_per_mwh": [float(today[market].min())],
        "today_max_eur_per_mwh": [float(today[market].max())],
    }))

    print("🥧 Cheapest hours today")
    display(cheapest)

    print("🍰 Most expensive hours today")
    display(priciest)

# --- 7) Save to ../Data/gold.csv
out_path = "../Data/gold.csv"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
gold.to_csv(out_path, index=False)
print(f"✅ Saved GOLD to: {out_path}  ({len(gold):,} rows)")

,rows,cols,start,end,markets
0,32,8,2026-02-10 14:00:00,2026-02-11 21:00:00,8


,missing_share
belgium_be,0.0
denmark_dk,0.0
germany_de,0.0
great_britain_gb,0.0
great_britain_gb__epex,0.0
great_britain_gb__nordpool,0.0
netherlands_nl,0.0
norway_no,0.0


,belgium_be,denmark_dk,germany_de,great_britain_gb,great_britain_gb__epex,great_britain_gb__nordpool,netherlands_nl,norway_no
date_cet,,,,,,,,
2026-02-10 14:00:00,103.19,118.02,116.22,108.68,108.28,108.84,109.12,113.17
2026-02-10 15:00:00,112.55,130.20,128.12,111.38,111.38,111.38,119.64,124.07
2026-02-10 16:00:00,110.50,131.15,127.66,113.96,114.71,113.67,118.42,124.14
2026-02-10 17:00:00,119.26,138.66,135.24,120.55,120.56,120.55,126.72,131.54
2026-02-10 18:00:00,121.80,136.04,131.71,123.04,123.09,123.03,126.35,130.60


C:\Users\joeri\AppData\Local\Temp\ipykernel_41584\3030508343.py:47: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\joeri\AppData\Local\Temp\ipykernel_41584\3030508343.py:58: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\joeri\AppData\Local\Temp\ipykernel_41584\3030508343.py:75: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\joeri\AppData\Local\Temp\ipykernel_41584\3030508343.py:86: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\joeri\AppData\Local\Temp\ipykernel_41584\3030508343.py:96: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\joeri\AppData\Local\Temp\ipykernel_41584\3030508343.py:108: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


,today_date,selected_market,today_avg_eur_per_mwh,today_min_eur_per_mwh,today_max_eur_per_mwh
0,2026-02-11,netherlands_nl,101.365909,82.88,121.65


🥧 Cheapest hours today


,netherlands_nl
date_cet,
2026-02-11 03:00:00,82.88
2026-02-11 04:00:00,83.01
2026-02-11 02:00:00,85.12
2026-02-11 00:00:00,87.17
2026-02-11 01:00:00,87.25


🍰 Most expensive hours today


,netherlands_nl
date_cet,
2026-02-11 08:00:00,121.65
2026-02-11 09:00:00,120.26
2026-02-11 18:00:00,115.32
2026-02-11 17:00:00,113.98
2026-02-11 07:00:00,113.90


✅ Saved GOLD to: ../Data/gold.csv  (32 rows)
